In [1]:
# Import the functions from /tools
import math
import os


from tools.db_detail import db_getdesc
from tools.db_detail import db_getnames
from tools.sql_execute import sqlite_execute as execute
from tqdm import tqdm
import json
import argparse

def sql_evoke(query,db_name):
    result, execution_time ,executable = execute("datasets/cosql_dataset/database/"+db_name+"/"+db_name+".sqlite",query)
    return result 

def get_example(db_name):
    sql_query = "SELECT name FROM sqlite_master WHERE type='table';"
    result = sql_evoke(sql_query,db_name)
    column_example=""
    for table_name in result:
        column_example = column_example + table_name[0] + ":\n"
        sql_get_eg = "SELECT * FROM "+ table_name[0] +" LIMIT 3;"
        table_eg = sql_evoke(sql_get_eg,db_name)
        for table_data in table_eg:
            column_example = column_example + '('
            for column_data in table_data: 
                column_example = column_example + str(column_data) +','
            column_example = column_example[:-1] + ')\n'
    return column_example

# print(db_getdesc("car_1"),get_example("car_1"))

In [2]:
from agents.rewriter import Rewriter
from agents.selector import Selector
from agents.detector import Detector
from agents.decomposer import Decomposer
from agents.refiner import Refiner

# creat and name the agents
rewriter = Rewriter("Rewriter")
selector = Selector("Selector")
detector = Detector("Detector")
decomposer = Decomposer("Decomposer")
refiner = Refiner("Refiner")

# input_data = "hey!"
# output_rewriter = rewriter.process_input(input_data)
# output_selector = selector.process_input(input_data)
# output_detector = detector.process_input(input_data)
# output_decomposer = decomposer.process_input(input_data)
# output_refiner = refiner.process_input(input_data)

In [3]:


def solve_answerable(db_name, output_selector, question):
    output_decomposer = ""
    output_refiner = ""
    
    print("________Decompose question and solve________")
    input_data = {
    "evidence": "",
    "db_name": db_name,
    "mini_schema": output_selector,
    "question": question,
    }

    output_decomposer = decomposer.process_input(input_data)
    print(output_decomposer)
    
    if output_decomposer["executable"]:
        return output_decomposer["sql"], output_decomposer, output_refiner
        # sql_output = " Result:" + str(output_decomposer["result"])
    else:
        
        print("________Refines erroneous SQL queries________")
        input_data = {
        "evidence": "",
        "db_name": db_name,
        "mini_schema": output_selector,
        "question": question,
        "old_sql": output_decomposer.get("sql"),
        "log": output_decomposer.get("log")
        }
        
        
        output_refiner = refiner.process_input(input_data)
        # print(output_refiner)
        print(output_refiner)
        return output_refiner["sql"], output_decomposer, output_refiner
        
        # if output_decomposer["executable"]:
        #     sql_output = " Result:" + str(output_refiner["result"])



def process_json_part(data, output_file):
    for index1,item in enumerate(tqdm(data)):
        retries = 0
        while retries < 2:
            try:
                # print(item)
                # Initialnize messages
                id_now = item.get('id')
                print("id " + str(id_now) + " Turn "+str(index1)+" ==================================================================")
                db_name = item['db_name']
                previous_QA = ""
                print(id_now)
                
                for index, turn in enumerate(item['turns']):
                    
                    if turn['isuser']:
                        final_output = ""
                        output_rewriter = ""
                        output_selector = ""
                        output_detector = ""
                        output_decomposer = ""
                        output_refiner = ""
                        final_output = ""
                        rewritten_questions = []
                        rewritten_outputs = []
                        sql_output = ""
                        user_question = turn['text']
                        question_type = turn['type']
        
                        print("previous_QA " + ":" + previous_QA)
                        print("question " + str(index//2) + ":\033[93m " + user_question + "\033[0m")
                        
                        if index+1<len(item['turns']):
        
                            # print("________Question rewriter________")
                            # input_data = {
                            #     "db_desc": db_getnames(db_name),
                            #     "evidence": "",
                            #     "question": user_question,
                            #     "previous_QA": previous_QA
                            # }
                            
                            # output_rewriter = rewriter.process_input(input_data)
                            
                            # if output_rewriter["improper"] == "YES":
                            #     final_output = output_rewriter["text"]

                            if user_question == "":
                                continue
                            
                            else:
                                # rewritten_question = output_rewriter["text"]
                                rewritten_question = "previous QA:" + previous_QA + "\ncurrent question:" + user_question
                                
                                print("________Select columns and values________")
                                input_data = {
                                    "evidence": "",
                                    "db_name": db_name,
                                    "db_desc": db_getdesc(db_name),
                                    "db_exam": get_example(db_name),
                                    "question": rewritten_question 
                                }
                                
                                # output_selector = selector.process_input(input_data)
                                output_selector = db_getdesc(db_name) + "Value examples:" + get_example(db_name)
                                print(output_selector)
                            
                                
                                print("________Question type detect________")
                                input_data = {
                                    "evidence": "",
                                    "db_name": db_name,
                                    "db_desc": db_getdesc(db_name),
                                    "mini_schema": output_selector,
                                    "question": rewritten_question
                                }
                                
                                # output_detector = detector.process_input(input_data)
                                # if output_detector == "YES":

                                
                                if "YES" == "YES":
                                    final_output, output_decomposer, output_refiner = solve_answerable(db_name, output_selector, rewritten_question)
                                    
                                else:
                                        
                                    final_output = output_detector.get("answer",output_detector)

                                    if output_detector.get("type",'').lower() != 'improper':
                                        rewritten_questions = output_detector.get("rewrite",[])
                                        rewritten_questions.append(user_question)
                                        print("rewritten_questions: :\033[94m" + str(rewritten_questions) + "\033[0m")
                                        
                                        for rewritten_question in rewritten_questions:
                                            possible_output, output_decomposer, output_refiner = solve_answerable(db_name, output_selector, "previous QA:" + previous_QA + "\ncurrent question:" + rewritten_question)
                                            if possible_output != "":
                                                rewritten_outputs.append(possible_output)
                                                            
                            # llm record
                            print("\nFINAL Response:")
                            print(final_output)
                            
                            item['turns'][index+1]['predict'] = final_output
                            item['turns'][index+1]['rewritten_questions'] = rewritten_questions
                            item['turns'][index+1]['rewritten_outputs'] = rewritten_outputs

                            
                            
                            item['turns'][index+1]['Rewriter'] = output_rewriter
                            item['turns'][index+1]['Selector'] = output_selector
                            item['turns'][index+1]['Detector'] = output_detector
                            item['turns'][index+1]['Decomposer'] = output_decomposer
                            item['turns'][index+1]['Refiner'] = output_refiner
                            
                            # update messages
                            g_ans = ""
                            if item['turns'][index+1]['text']:
                                g_ans = item['turns'][index+1]['text']
                            else:
                                g_ans = item['turns'][index+1]['query']
                            if question_type == "answerable":
                                sql_result = sql_evoke(g_ans,db_name)
                                sql_output = " Result:" + str(sql_result)
                            # previous_QA += "\nQ:" + user_question + "\nA:" + g_ans + sql_output + '\n'
                            previous_QA += "\nQ:" + user_question + "\nA:" + g_ans + '\n'
                            print("gold answer: " + g_ans)
                            print("————————————————————————————————————————————————————————————")
                            
                if not os.path.exists(output_file):
                    with open(output_file, 'w') as f:
                        items = [item]
                        json.dump(items, f, indent=4)
                        f.write('\n')
                else:
                    with open(output_file, 'r') as f:
                        try:
                            items = json.load(f)
                        except json.JSONDecodeError:
                            print("\033[91mError:The file content is not in valid JSON format\033[0m")
                            # items = []
        
                    if not isinstance(items, list):
                        print("\033[91mError:The file content is not in valid JSON format\033[0m")
                        # items = []

                    items.append(item)

                    with open(output_file, 'w') as f:
                        json.dump(items, f, indent=4)
                        f.write('\n')
                break
            except Exception as e:
                retries += 1
                print(f"\033[91m==============Error processing id {id_now} data index {index} (attempt {retries}): {e}==============\033[0m")

with  open('datasets/MMSQL_test.json', 'r', encoding='utf-8') as f:
        data = json.load(f)



    
# process_json_part(data[55:58],'text.json')

indices = [138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74] 
extracted_data = [data[i] for i in indices]
process_json_part(extracted_data,"gemini-wo-detector.json")

  0%|                                                                                                                                                                                                                                                                                              | 0/28 [00:00<?, ?it/s]

id 138 Turn 0 ==================================================================
138
previous_QA :
question 0: Show the names of all of the high schoolers.
________Select columns and values________
Highschooler(ID:id type:number PRIMARY KEY|name:name type:text|grade:grade type:number|)
Friend(student_id:student id type:number PRIMARY KEY|friend_id:friend id type:number|)
Likes(student_id:student id type:number PRIMARY KEY|liked_id:liked id type:number|)
Foreign keys:
Friend.'friend_id' = Highschooler.'ID'
Friend.'student_id' = Highschooler.'ID'
Likes.'student_id' = Highschooler.'ID'
Likes.'liked_id' = Highschooler.'ID'
Value examples:Highschooler:
(1510,Jordan,9)
(1689,Gabriel,9)
(1381,Tiffany,9)
Friend:
(1510,1381)
(1510,1689)
(1689,1709)
Likes:
(1689,1709)
(1709,1689)
(1782,1709)

________Question type detect________
________Decompose question and solve________
request gemini-1.5-flash
HTTPSConnectionPool(host='generativelanguage.googleapis.com', port=443): Max retries exceeded with 

  4%|█████████▉                                                                                                                                                                                                                                                                            | 1/28 [01:29<40:06, 89.13s/it]

Success: Operation completed after 0 retries
{'result': '', 'sql': 'improper  ', 'executable': True, 'log': '', 'process': 'improper \n'}

FINAL Response:
improper  
gold answer: Thank you. Bye
————————————————————————————————————————————————————————————
id 139 Turn 1 ==================================================================
139
previous_QA :
question 0: Hi! How many owners temporarily do not have dogs?
________Select columns and values________
Breeds(breed_code:breed code type:text PRIMARY KEY|breed_name:breed name type:text|)
Charges(charge_id:charge id type:number PRIMARY KEY|charge_type:charge type type:text|charge_amount:charge amount type:number|)
Sizes(size_code:size code type:text PRIMARY KEY|size_description:size description type:text|)
Treatment_Types(treatment_type_code:treatment type code type:text PRIMARY KEY|treatment_type_description:treatment type description type:text|)
Owners(owner_id:owner id type:number PRIMARY KEY|first_name:first name type:text|last_name:

  7%|███████████████████▊                                                                                                                                                                                                                                                                  | 2/28 [01:48<20:50, 48.11s/it]

...

id 73 Turn 26 ==================================================================
73
previous_QA :
question 0: Which model saves the most gasoline? That is to say, have the maximum miles per gallon?
________Select columns and values________
continents(ContId:cont id type:number PRIMARY KEY|Continent:continent type:text|)
countries(CountryId:country id type:number PRIMARY KEY|CountryName:country name type:text|Continent:continent type:number|)
car_makers(Id:id type:number PRIMARY KEY|Maker:maker type:text|FullName:full name type:text|Country:country type:text|)
model_list(ModelId:model id type:number PRIMARY KEY|Maker:maker type:number|Model:model type:text|)
car_names(MakeId:make id type:number PRIMARY KEY|Model:model type:text|Make:make type:text|)
cars_data(Id:id type:number PRIMARY KEY|MPG:mpg type:text|Cylinders:cylinders type:number|Edispl:edispl type:number|Horsepower:horsepower type:text|Weight:weight type:number|Accelerate:accelerate type:number|Year:year type:number|)
Foreign k

 96%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████          | 27/28 [10:31<00:11, 11.13s/it]

Success: Operation completed after 0 retries
{'result': [('ford',)], 'sql': 'SELECT T1.Model FROM CAR_NAMES AS T1 JOIN CARS_DATA AS T2 ON T1.MakeId = T2.Id ORDER BY T2.Horsepower DESC LIMIT 1', 'executable': True, 'log': '', 'process': 'Subquestion 1: Which model has the highest horsepower? \nSQL ```sql SELECT T1.Model FROM CAR_NAMES AS T1 JOIN CARS_DATA AS T2 ON T1.MakeId = T2.Id ORDER BY T2.Horsepower DESC LIMIT 1```\n\nQuestion Solved. \n'}

FINAL Response:
SELECT T1.Model FROM CAR_NAMES AS T1 JOIN CARS_DATA AS T2 ON T1.MakeId = T2.Id ORDER BY T2.Horsepower DESC LIMIT 1
gold answer: SELECT T1.Model FROM CAR_NAMES AS T1 JOIN CARS_DATA AS T2 ON T1.MakeId  =  T2.Id ORDER BY T2.horsepower DESC LIMIT 1;
————————————————————————————————————————————————————————————
id 74 Turn 27 ==================================================================
74
previous_QA :
question 0: how many players are presented on the table
________Select columns and values________
players(player_id:player id type

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 28/28 [11:15<00:00, 24.13s/it]

Success: Operation completed after 0 retries
{'result': '', 'sql': 'improper  ', 'executable': True, 'log': '', 'process': 'improper \n'}

FINAL Response:
improper  
gold answer: Bye
————————————————————————————————————————————————————————————


In [4]:
import json

def check_id_range(json_data):
  """
  Checks if the id field in the JSON data contains all numbers from 0 to 148.

  Args:
    json_data: JSON data containing an 'id' field.

  Returns:
    True if all numbers from 0 to 148 are present in the 'id' field, False otherwise.
  """
    
  id_set = set([item['id'] for item in json_data])
  expected_ids = set(range(149))  # Create a set containing numbers 0-148
  missing_ids = expected_ids - id_set
  if missing_ids:
    print("Missing IDs:", missing_ids)
  else:
    print("All IDs are present.")

# Load JSON data from a file, or define a JSON string directly
json_file_path = "gemini-wo-detector.json"  # Replace with your JSON file path
with open(json_file_path, 'r', encoding='utf-8') as f:
  json_data = json.load(f)

# Check ID range
check_id_range(json_data)

All IDs are present.
